In [5]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_261"; Java(TM) SE Runtime Environment (build 1.8.0_261-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.261-b12, mixed mode)
  Starting server from /usr/local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/_k/sjnxlkhj3vv__mxt3h2m8v3r0000gn/T/tmpv9wdpygr
  JVM stdout: /var/folders/_k/sjnxlkhj3vv__mxt3h2m8v3r0000gn/T/tmpv9wdpygr/h2o_spuebla_started_from_python.out
  JVM stderr: /var/folders/_k/sjnxlkhj3vv__mxt3h2m8v3r0000gn/T/tmpv9wdpygr/h2o_spuebla_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.6
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_spuebla_72oqzy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.768 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [12]:
import pandas as pd

In [13]:
train=pd.read_csv("input/train_diam.csv")
test=pd.read_csv("input/test_diam.csv")

In [14]:
h2train=h2o.H2OFrame(train)
h2test=h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [58]:
h2train.columns

['carat',
 'x',
 'y',
 'z',
 'price',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2']

In [59]:
predictors=['carat',
 'x',
 'y',
 'z',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2']

In [60]:
response="price"

In [61]:
train, valid=h2train.split_frame(ratios=[0.8],seed=1234)

In [62]:
diam_gbm = H2OGradientBoostingEstimator(nfolds=20,
                                        seed=1111,
                                        keep_cross_validation_predictions = True)

In [63]:
diam_gbm.train(x=predictors, y=response, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [64]:
perf = diam_gbm.model_performance()

In [65]:
pred = diam_gbm.predict(valid)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [67]:
perf


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 412409.1867300371
RMSE: 642.1909270069432
MAE: 353.79289235597395
RMSLE: 0.14010715595934192
Mean Residual Deviance: 412409.1867300371


In [27]:
pred

predict
891.277
891.476
5674.49
9232.75
2558.73
14028
701.663
2028.06
733.486
2297.16


In [94]:
pred = pros_gbm.predict(h2test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [34]:
test["price"]

AttributeError: 'H2OFrame' object has no attribute 'T'

In [95]:
type(pred)

h2o.frame.H2OFrame

In [96]:
final=pred.as_data_frame()

In [97]:
final.rename(columns={"predict":"price"}, inplace=True)
final.index.rename("id", inplace=True)
final

,price
id,
0,15581.486845
1,15158.938773
2,1276.386714
3,489.902586
4,3557.533068
...,...
13480,2088.518744
13481,5109.574217
13482,2751.061819


In [98]:
final.to_csv("output/test2.csv")